In [49]:
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#creating list of file paths
folder = './data/'
all_files = glob.glob(os.path.join(folder, "*.csv"))

In [29]:
# importing all files into dataframes within a list
dataframes = []
tables = []

for file in all_files: 
    df = pd.read_csv(file,sep=';', parse_dates=['timestamp'])
    file = file.split("/")[-1].split('.')[0]
    dataframes.append(df)
    tables.append(file)

In [30]:
df = pd.concat(dataframes)

In [39]:
df.set_index('timestamp', inplace=True)